# AB SEO Test Summary

In [4]:
from athena_querying import *

import pandas as pd
import numpy as np
from datetime  import datetime, timedelta, date
import time

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

## Settings

# Get Results

In [11]:

athena_conn = connect_to_athena()

In [12]:
print("Checking athena works")
query = "select * from ms_data_stream_production_processed limit 100"
test_query_results = pd.read_sql(query, athena_conn)

test_query_results.head(10)

Checking athena works


,type,context,user,body,sent_at
0,page,{title=Intip Rumah Annisa Pohan dan AHY yang M...,{anonymous_id=1562c173-aec0-4ce2-976f-b3c9fbbe...,"{event_name=PageView, data={title=null, status...",2019-08-04T01:24:12.775Z
1,event,{title=Alasan Sulit Naik Jabatan di Tempat Ker...,{anonymous_id=c60dd66e-9a42-44ca-b071-1f15801c...,"{event_name=Reading, data={title=Alasan Sulit ...",2019-08-04T01:24:13.847Z
2,event,{title=Bisnis Makanan Ringan yang Bisa Jalan d...,{anonymous_id=ab992af9-b922-4e05-bbb1-14199db9...,"{event_name=Reading, data={title=Bisnis Makana...",2019-08-04T01:24:13.619Z
3,event,{title=5 Cara Keluarga Ruben Onsu Ngabisin Dui...,{anonymous_id=7174c4ef-ec98-4e92-bdde-65c671c0...,"{event_name=Reading, data={title=5 Cara Keluar...",2019-08-04T01:24:13.606Z
4,event,{title=Orang Sukses yang Introvert dengan Tota...,{anonymous_id=17167f9e-e9b9-47c1-a2f8-5934d24b...,"{event_name=Reading, data={title=Orang Sukses ...",2019-08-04T01:24:13.885Z
5,event,{title=Rumah Agus Surya Bakti dan Bella Saphir...,{anonymous_id=751b6784-91d9-4fec-bf17-764af7b5...,"{event_name=Reading, data={title=Rumah Agus Su...",2019-08-04T01:24:08.457Z
6,event,{title=Jedar Nginap di Ritz Carlton Pacific Pl...,{anonymous_id=ae5185c0-e0b3-484d-af05-748f468b...,"{event_name=Reading, data={title=Jedar Nginap ...",2019-08-04T01:24:14.162Z
7,event,"{title=16 Perusahaan Bonafit di Dunia, Berapa ...",{anonymous_id=bdbd40ee-4640-4db8-a213-abb59c71...,"{event_name=Reading, data={title=16 Perusahaan...",2019-08-04T01:24:14.381Z
8,event,{title=Andhika Pratama dan Ussy Sulistyawati T...,{anonymous_id=2dc8812b-63f3-43ab-abc4-e36707ca...,"{event_name=Reading, data={title=Andhika Prata...",2019-08-04T01:24:13.334Z
9,event,"{title=Rumah Iis Dahlia Harganya Rp 17 Miliar,...",{anonymous_id=af7d3b7f-0c5c-4b5e-b014-8be400fd...,"{event_name=Reading, data={title=Rumah Iis Dah...",2019-08-04T01:24:14.940Z


In [ ]:
# blog_pageview_metrics
# blog_pageviews_per_user_per_day

In [10]:
query = "select * from blog_pageview_metrics order by day_str, is_test"
pageview_metrics_results = pd.read_sql(query, athena_conn)
pageview_metrics_results.head()

DatabaseError: Execution failed on sql: select * from blog_pageview_metrics order by day_str, is_test
SYNTAX_ERROR: line 1:15: Table awsdatacatalog.default.blog_pageview_metrics does not exist
unable to rollback